# Turbulence parameterizations

The cornerstone of parameterizing turbulence in weather and climate models is Monin-Obukhov similarity theory (MOST, @Monin1954). This theory is derived from the TKE equation, assuming stationarity and homogeneity (and neglecting pressure fluctuations), and states that all dimensionless turbulent quantities can be expressed in terms of the stability parameter $\zeta$. This has the consequence that subgrid-scale quantities, such as fluxes or turbulence intensities, can be parameterized (after normalization) in terms of static stability. Two MOST methods to parameterize fluxes are (1) flux-variance relations (used mainly in dispersion models) and (2) flux-profile relations (used in weather and climate models).

## Flux-variance relations
Flux-variance relations follow from MOST, and relate the (temporal) variance ($\sigma_x$) of a scalar ($x$) to its vertical flux ($\overline{w'x'}$) by

$$ \frac{\sigma_x}{x_*} = \Phi_x(\zeta) \quad \quad \textrm{with} \quad\quad x_* := \frac{\overline{w'x'}}{u_*}.  $$
Thereby, $\Phi_x(\zeta)$ is a stability correction function that accounts for diabatic processes. Under neutral conditions, the so called integral turbulence quantities, i.e. $\sigma_x/u_*$, are constant, so that there no stability correction is needed. 
When measuring fluxes and variances with the eddy-covariance method, the function $\Phi_x(\zeta)$ can be fitted to the observations, and are assumed to have a certain "universality", so that they are transferable to other sites. This offers the possibility to then estimate the flux from the variance (and friction velocity and stability) by
$$ \overline{w'x'} = \frac{\sigma_x \, u_*}{\Phi_x(\zeta)}. $$

Eventhough stability correction functions show some "universality", recent studies found the spread around stability correction function can be partly attributed to turbulence anisotropy (@Stiperski2018) and flow organization (@Mack2024), indicating that surface and flow properties are further important scaling parameters (additionally to $\zeta$), particularly over complex terrain.

## Flux-profile relations

## Using NWP model output
